# Preprocessing 
Using the preprocess script and saving the processed file in new location

In [73]:
path2csv = "data/marketing_campaign.csv"
path2processedcsv = "data/processed_marketing_campaign.csv"

In [74]:
!python preprocess.py \
    --pathToCSV data/marketing_campaign.csv \
    --pathToSaveProcessedCSV data/processed_marketing_campaign.csv

---Started Preprocessing---


Dropping rows having null values
Extracting new feature from Birth Year feature
     ID  Year_Birth   Education  ... Z_Revenue  Response  Age
0  5524        1957  Graduation  ...        11         1   64
1  2174        1954  Graduation  ...        11         0   67
2  4141        1965  Graduation  ...        11         0   56
3  6182        1984  Graduation  ...        11         0   37
4  5324        1981         PhD  ...        11         0   40

[5 rows x 30 columns]


 Creating new feature total spent from spent features 
     ID  Year_Birth   Education  ... Response  Age  TotalSpent
0  5524        1957  Graduation  ...        1   64        1617
1  2174        1954  Graduation  ...        0   67          27
2  4141        1965  Graduation  ...        0   56         776
3  6182        1984  Graduation  ...        0   37          53
4  5324        1981         PhD  ...        0   40         422

[5 rows x 31 columns]

 
Extracing Months spent on the plat

# Exploratory Data Analysis

In [60]:
import pandas as pd

In [61]:
df = pd.read_csv(path2processedcsv)

In [62]:
df.head()

,Education,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Age,TotalSpent,Mn_Customer,AgeGroup,Children,campaignConvert
0,-0.790438,0.287105,-0.822754,-0.929699,0.310353,0.977660,1.552041,1.690293,2.453472,1.483713,...,1.426865,2.503607,-0.555814,0.692181,1.018352,1.676245,1.936840,1.237486,-1.264598,1.0
1,-0.790438,-0.260882,1.040021,0.908097,-0.380813,-0.872618,-0.637461,-0.718230,-0.651004,-0.634019,...,-1.126420,-0.571340,-1.171160,-0.132545,1.274785,-0.963297,-1.714136,1.237486,1.404572,0.0
2,-0.790438,0.913196,-0.822754,-0.929699,-0.795514,0.357935,0.570540,-0.178542,1.339513,-0.147184,...,1.426865,-0.229679,1.290224,-0.544908,0.334530,0.280110,-0.149432,-0.221639,-1.264598,0.0
3,-0.790438,-1.176114,1.040021,-0.929699,-0.795514,-0.872618,-0.561961,-0.655787,-0.504911,-0.585335,...,-0.761665,-0.913000,-0.555814,0.279818,-1.289547,-0.920135,-1.974920,-1.680765,0.069987,0.0
4,1.590024,0.294307,1.040021,-0.929699,1.554453,-0.392257,0.419540,-0.218684,0.152508,-0.001133,...,0.332600,0.111982,0.059532,-0.132545,-1.033114,-0.307562,-0.801392,-0.221639,0.069987,0.0


In [63]:
df.iloc[:, 1]

0       0.287105
1      -0.260882
2       0.913196
3      -1.176114
4       0.294307
          ...   
2207    0.430444
2208    0.560123
2209    0.233347
2210    0.803172
2211    0.042290
Name: Income, Length: 2212, dtype: float64

In [64]:
df.Income.describe()

count    2.212000e+03
mean    -1.252765e-16
std      1.000226e+00
min     -2.333791e+00
25%     -7.771113e-01
50%     -2.731156e-02
75%      7.679525e-01
max      5.131311e+00
Name: Income, dtype: float64

In [65]:
df.campaignConvert

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2207    1.0
2208    0.0
2209    0.0
2210    0.0
2211    0.0
Name: campaignConvert, Length: 2212, dtype: float64

In [66]:
df.columns

Index(['Education', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines',
       'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Age',
       'TotalSpent', 'Mn_Customer', 'AgeGroup', 'Children', 'campaignConvert'],
      dtype='object')

In [72]:
import xgboost as xgb

X, y = df.iloc[:,df.columns != 'campaignConvert'], df['campaignConvert']
    # Specify `enable_categorical` to True.
reg = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)
reg.fit(X, y, eval_set=[(X, y)])

# Pass in already encoded data
X_enc, y_enc = make_categorical(100, 10, 4, True)
reg_enc = xgb.XGBRegressor(tree_method="gpu_hist")
reg_enc.fit(X_enc, y_enc, eval_set=[(X_enc, y_enc)])

reg_results = np.array(reg.evals_result()["validation_0"]["rmse"])
reg_enc_results = np.array(reg_enc.evals_result()["validation_0"]["rmse"])

# Check that they have same results
np.testing.assert_allclose(reg_results, reg_enc_results)

# Convert to DMatrix for SHAP value
booster: xgb.Booster = reg.get_booster()
m = xgb.DMatrix(X, enable_categorical=True)  # specify categorical data support.
SHAP = booster.predict(m, pred_contribs=True)
margin = booster.predict(m, output_margin=True)
np.testing.assert_allclose(
    np.sum(SHAP, axis=len(SHAP.shape) - 1), margin, rtol=1e-3
)

AttributeError: module 'xgboost' has no attribute 'XGBRegressor'